In [1]:
!pip install langchain faiss-gpu jq
!pip install accelerate
!pip install transformers
!pip install -U langchain-community
!pip install tiktoken
!pip install sentence-transformers

In [2]:
!pip install tensorflow[and-cuda]
!pip install torch~=2.3.0 torch_xla[tpu]~=2.3.0 -f https://storage.googleapis.com/libtpu-releases/index.html

  Using cached nvidia_cublas_cu12-12.3.4.1-py3-none-manylinux1_x86_64.whl (412.6 MB)
  Using cached nvidia_cuda_cupti_cu12-12.3.101-py3-none-manylinux1_x86_64.whl (14.0 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.3.107-py3-none-manylinux1_x86_64.whl (24.9 MB)
  Using cached nvidia_cuda_runtime_cu12-12.3.101-py3-none-manylinux1_x86_64.whl (867 kB)
  Using cached nvidia_cudnn_cu12-8.9.7.29-py3-none-manylinux1_x86_64.whl (704.7 MB)
  Using cached nvidia_cufft_cu12-11.0.12.1-py3-none-manylinux1_x86_64.whl (98.8 MB)
  Using cached nvidia_curand_cu12-10.3.4.107-py3-none-manylinux1_x86_64.whl (56.3 MB)
  Using cached nvidia_cusolver_cu12-11.5.4.101-py3-none-manylinux1_x86_64.whl (125.2 MB)
  Using cached nvidia_cusparse_cu12-12.2.0.103-py3-none-manylinux1_x86_64.whl (197.5 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.20.5
    Uninstalling nvidia-nccl-cu12-2.20.

In [3]:
from langchain import HuggingFacePipeline
from accelerate import Accelerator
from langchain.text_splitter import TokenTextSplitter,CharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

In [12]:
import os
from huggingface_hub import notebook_login
os.environ['HUGGINGFACEHUB_API_TOKEN']='YOUR_HF_API_TOKEN'
notebook_login()

In [5]:
# Do not run this cell if not using Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["year"] = record.get("pub_date").get('year')
    metadata["month"] = record.get("pub_date").get('month')
    metadata["day"] = record.get("pub_date").get('day')
    metadata["title"] = record.get("article_title")
    return metadata

In [7]:
from langchain.document_loaders import JSONLoader

# Set file path to JSON files
loader1 = JSONLoader(
    file_path='/content/drive/MyDrive/pubmed_background.json',
    jq_schema='.[]',
    content_key='article_abstract',
    metadata_func=metadata_func)

loader2 = JSONLoader(
    file_path='/content/drive/MyDrive/pubmed_reference.json',
    jq_schema='.[]',
    content_key='article_abstract',
    metadata_func=metadata_func)

data1 = loader1.load()
data2 = loader2.load()

In [8]:
print(f"{len(data1)} pubmed articles are loaded!")
print(f"{len(data2)} pubmed articles are loaded!")

728 pubmed articles are loaded!
8321 pubmed articles are loaded!


In [9]:
text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=50)

chunks1 = text_splitter.split_documents(data1)
print(f"{len(chunks1)} chunks are created!")

chunks2 = text_splitter.split_documents(data2)
print(f"{len(chunks2)} chunks are created!")

1575 chunks are created!
16494 chunks are created!


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

modelPath = "intfloat/e5-large-unsupervised"

embeddings = HuggingFaceEmbeddings(
  model_name = modelPath,
  model_kwargs = {'device':'cpu'},
  encode_kwargs={'normalize_embeddings':False})

# Using faiss index
from langchain.vectorstores import FAISS
background_db = FAISS.from_documents(chunks1, embeddings)
reference_db = FAISS.from_documents(chunks2, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [13]:
import torch

# Set model from huggingface directory
model_id = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id,offload_buffers=True)
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=False, device_map='auto')

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1048)
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0.5}
)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [18]:
# Define the langchain pipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

BACKGROUND_TEMPLATE = """
    You are a medical research assistant to help write an academic research paper.
    Construct a five paragraph essay Outline about the Topic using the provided Context only.
    Your outline should be in an academic tone.
    Context: {context}
    Topic: {question}
    Outline:
    """
BACKGROUND_PROMPT = PromptTemplate.from_template(BACKGROUND_TEMPLATE)

# Develop topic sentences and outline from background information
background_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=background_db.as_retriever(k=2),
    chain_type_kwargs={"prompt": BACKGROUND_PROMPT},
    return_source_documents=True
)

In [22]:
# Make outline based on a topic sentence
query = "The topic is COVID-19 and the symptoms associated with it."
result = background_chain({"query": query})
print(result['result'].strip())

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You are a medical research assistant to help write an academic research paper.
    Construct a five paragraph essay Outline about the Topic using the provided Context only.
    Your outline should be in an academic tone.
    Context:  investigate and quantify the negative impact that the COVID-19 pandemic has industry-sponsored clinical trials, both in the USA and worldwide. We find a negative correlation between the severity of the COVID-19 pandemic and clinical trial screening rate, with the relationship being strongest during the first three months of the pandemic compared to the entire duration of the pandemic. This negative statistical relationship holds across therapeutic areas, across states in the USA despite the heterogeneity of responses at the state-level, and across countries. This work has significant implications for the management of clinical trials worldwide in response to the fluctuating severity of COVID-19 moving

 highlighted that no official guidelines exist to app

In [23]:
# Topic Sentence Sources
titles = ['\t-'+doc.metadata['title'] for doc in result['source_documents']]
print("\n\nThe provided answer is based on the following PubMed articles:\t")
print("\n".join(set(titles)))



The provided answer is based on the following PubMed articles:	
	-Curious Questions About Covid-19 Pandemic with ChatGPT: Answers and Recommendations.
	-Quantifying the impact of the COVID-19 pandemic on clinical trial screening rates over time in 37 countries.
	-The definition of long COVID used in interventional studies.
	-Down syndrome and biological treatments in dermatology: Efficacy and safety in our real-life experience and review of literature.


In [24]:
REFERENCE_TEMPLATE = """
    You are a medical research assistant to help write an academic research paper.
    Answer the Question using the provided Context only.
    Your answer should be based on the Outline provided, written in an academic tone and consist of at least 5 paragraphs.
    Context: {context}
    Outline: {question}
    Answer:
    """
REFERENCE_PROMPT = PromptTemplate.from_template(REFERENCE_TEMPLATE)

# Produce an essay of at least 5 paragraphs from reference information
reference_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=reference_db.as_retriever(k=2),
    chain_type_kwargs={"prompt": REFERENCE_PROMPT},
    return_source_documents=True
)

In [25]:
# Copy and paste outline generated by background_chain into the query for reference_chain to write essay
query = """
        Outline:
        1. Introduction: Provide an overview of the topic and the research question.
        2. Literature Review: Review the existing literature on the topic and identify any gaps in knowledge.
        3. Methods: Describe the methods used to collect and analyze data.
        4. Results: Present the results of the data analysis and discuss their implications.
        5. Discussion: Interpret the results in light of the existing literature and identify any limitations of the study.
        6. Conclusion: Summarize the main findings of the study and discuss their implications for future research.
        """
result = reference_chain({"query": query})
print(result['result'].strip())

# Metadata
titles = ['\t-'+doc.metadata['title'] for doc in result['source_documents']]
print("\n\nThe provided answer is based on the following PubMed articles:\t")
print("\n".join(set(titles)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


You are a medical research assistant to help write an academic research paper.
    Answer the Question using the provided Context only.
    Your answer should be based on the Outline provided, written in an academic tone and consist of at least 5 paragraphs.
    Context: This was a primary research study.

Thank you kindly for your interest in and opinion [...].

i>P</i> = 0.034). The sleep quality score after the intervention was significantly lower in the intervention group than in the control group. This indicates better sleep quality in the intervention group. The results also show that the level of visual fatigue in the intervention group decreased significantly. However, no significant change was detected regarding positive and negative emotions. After the intervention, cortisol levels were significantly higher in the intervention group than the control group. In addition, cortisol levels increased significantly and melatonin levels decreased significantly in the intervention gro

In [26]:
# Paragraph Sources
titles = ['\t-'+doc.metadata['title'] for doc in result['source_documents']]
print("\n\nThe provided answer is based on the following PubMed articles:\t")
print("\n".join(set(titles)))



The provided answer is based on the following PubMed articles:	
	-Feasibility and Initial Efficacy of an Adapted Telepractice Listening Comprehension Intervention for School-Aged Children with Autism.
	-A New Predictive Nomogram for the Risk of Delayed Incision Healing After Open Posterior Lumbar Surgery: A Retrospective Study.
	-Reply to Lee, S.Y. Comment on "Sung et al. Body Fat Reduction Effect of <i>Bifidobacterium breve</i> B-3: A Randomized, Double-Blind, Placebo Comparative Clinical Trial. <i>Nutrients</i> 2023, <i>15</i>, 28".
	-Investigating the effects of a blue-blocking software on the daily rhythm of sleep, melatonin, cortisol, positive and negative emotions.
